<a href="https://colab.research.google.com/github/olgasherbiena/KPI/blob/IDA/1_5_Data_visualization_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import kagglehub
from kagglehub import KaggleDatasetAdapter

## Load the Data

In [ ]:
import os

dataset_path = kagglehub.dataset_download("albertobircoci/support-ticket-priority-dataset-50k")
print("Path to dataset files:", dataset_path)

support_ticket_data = pd.read_csv(os.path.join(dataset_path, "Support_tickets.csv"))

In [ ]:
print(support_ticket_data.columns)

In [ ]:
df_support = support_ticket_data[['day_of_week_num', 'company_size_cat', 'industry_cat', 'region_cat', 'customer_tier_cat', 'customers_affected', 'priority_cat']]
feature_names = support_ticket_data.columns[:5]

df_support.info()

In [ ]:
# унікальні класи
print("Unique classes:", set(df_support['priority_cat']))  # або 'priority_cat' для числового варіанту
print(":", set(df_support['industry_cat']))
print(":", set(df_support['company_size_cat']))
print(":", set(df_support['day_of_week_num']))
print(":", set(df_support['region_cat']))
print("Unique classes:", set(df_support['customer_tier_cat']))  # або 'priority_cat' для числового варіанту
print("Unique classes:", set(df_support['customers_affected']))

# якщо хочеш "людинозрозумілі" назви класів
print("Class names:", support_ticket_data['priority_cat'].unique())
print("Class names:", support_ticket_data['industry_cat'].unique())
print("Class names:", support_ticket_data['company_size_cat'].unique())
print("Class names:", support_ticket_data['day_of_week_num'].unique())
print("Class names:", support_ticket_data['region_cat'].unique())
print("Class names:", support_ticket_data['customer_tier_cat'].unique())
print("Class names:", support_ticket_data['customers_affected'].unique())



# Pairwise correlation of columns

In [ ]:
corr = df_support.corr()
fig, ax = plt.subplots(figsize=(5, 4), dpi = 80)
sns.heatmap(corr,  cmap='coolwarm', annot=True, fmt=".2f")
plt.show()
# This heatmap shows the correlation between all numeric features in the wine dataset.
# Values close to 1 indicate a strong positive relationship, values close to -1 indicate a strong negative relationship,
# and values near 0 mean little or no linear correlation.
# Correlation with the target can be useful, because it shows which features are most related to the class labels.
#However, strong correlation between features themselves may cause multicollinearity,
#which can be a problem for some machine learning models.

## Boxplots
for more examples see https://seaborn.pydata.org/generated/seaborn.boxplot.html

In [ ]:
features_name = ['day_of_week_num', 'region_cat', 'customer_tier_cat', 'customers_affected']

for i in range(len(features_name)):
    plt.figure(figsize=(5, 4), dpi=80)
    ax = sns.boxplot(x='priority_cat', y=features_name[i], data=support_ticket_data)
    plt.title(f"Boxplot of {features_name[i]} by Priority")
    plt.show()


# Each boxplot compares the distribution of a single feature across the three wine classes.
# The box shows the interquartile range (IQR), the line inside is the median, and the whiskers/outliers indicate variability.
# If the boxes for different classes do not overlap much, it means that this feature is a good candidate for distinguishing between wine types.

## Pairplot
for more examples see
https://seaborn.pydata.org/generated/seaborn.pairplot.html

In [ ]:
plt.rcParams['figure.dpi'] = 70   # дефолт ~100, для зменшеня розміру зображення
sns.pairplot(df_support, hue="priority_cat", palette = 'Set1',  height=1.8)
plt.show()
#The pairplot shows scatterplots for every pair of features, colored by the wine class.
# This helps us visually check if classes are separable in 2D feature spaces and also reveals potential correlations or clusters.
# Diagonal plots show the distribution (histogram/density) of each single feature.

In [ ]:
plt.figure(figsize=(5,3))
sns.histplot(data=df_support, x="company_size_cat", hue="priority_cat", kde=True, multiple="stack", palette = 'Set2')
plt.show()
# This shows the distribution of one feature (alcohol) for each wine class.
# Overlaps indicate similarities, while clear separation suggests this feature helps classification.
# kde=True adds a smooth density curve on top of the histogram

In [ ]:
plt.figure(figsize=(5,3))
sns.violinplot(x="priority_cat", y="customers_affected", hue="priority_cat",
               data=df_support, palette="Set2")
plt.show()
#A violin plot combines a boxplot with a density curve, so you can see both the summary
# statistics and the shape of the distribution for each class.

# **K_Nearest_Neighbors**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display="text")

In [ ]:
support_ticket_data.head()

In [ ]:
support_ticket_data.info()

In [ ]:
#рахуємо к-сть пропущених даних
support_ticket_data.isnull().sum()

In [ ]:
columns_to_drop = [
    'ticket_id', 'day_of_week', 'company_id', 'company_size', 'industry',
    'customer_tier', 'region', 'product_area', 'booking_channel',
    'reported_by_role', 'priority', 'customer_sentiment', 'org_users', 'past_90d_incidents', 'past_30d_tickets',
]

support_ticket_data = support_ticket_data.drop(columns=columns_to_drop, axis=1)

In [ ]:
support_ticket_data['priority_cat'].describe()

In [ ]:
support_ticket_data.head()

In [ ]:
support_ticket_data.info()

# **Train Test Split**

In [ ]:
X = support_ticket_data.drop(['priority_cat'], axis = 1)
y = support_ticket_data['priority_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

Standardize the Variables
Алгоритми машинного навчання, що використовують відстані (наприклад KNN) чутливі до масштабу ознак

StandardScaler приводить всі ознаки до одного масштабу:

-середнє значення = 0, стандартне відхилення = 1

(щоб жодна змінна не «домінувала» тільки через свій масштаб)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

KNN model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15)

In [ ]:
knn.fit(scaled_X_train, y_train)

In [ ]:
pred = knn.predict(scaled_X_test)

Evaluation

In [ ]:
print(confusion_matrix(y_test,pred))

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
print(accuracy_score(y_test,pred))

Choosing a K Value

In [ ]:
error_rate = []

for i in range(1,30):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(scaled_X_train,y_train)
    pred_i = knn.predict(scaled_X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,30), error_rate, marker='o', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')
plt.grid()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

knn.fit(scaled_X_train,y_train)
pred = knn.predict(scaled_X_test)

print(confusion_matrix(y_test,pred))
print()
print(classification_report(y_test,pred))
print()
print(accuracy_score(y_test,pred))

Найважливіші параметри (коротко)

n_neighbors = 15 — кількість найближчих сусідів, які враховуються при класифікації

metric = 'minkowski' — метрика для обчислення відстаней

p = 2 — параметр для Minkowski: p=2 → евклідова відстань, p=1 → мангеттенська

weights ∈ {'uniform', 'distance'} — спосіб врахування сусідів: усі однаково чи ближчі мають більшу вагу, default=’uniform’

In [ ]:
knn.get_params()

# **Decision_Tree**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import set_config
set_config(display="text")

In [ ]:
support_ticket_data.head()

In [ ]:
support_ticket_data.columns

In [ ]:
print(type(X), type(y))
print(X.shape, y.shape)

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
# stratify=y: зберігає ті самі пропорції класів у train і test (стратифіковане розбиття)
# Особливо важливо для незбалансованих даних.

In [ ]:
print(type(X), type(y))
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Decision Tree
Note: scaling is NOT required for trees

Найважливіші параметри (коротко)

criterion={'gini','entropy'} — функція розщеплення

max_depth=None — максимальна глибина дерева (контроль пере/недонавчання)

min_samples_split=2 — мінімум зразків, щоб вузол ділився

min_samples_leaf=1 — мінімум зразків у листку

class_weight=None — корисно для незбалансованих даних ('balanced')

random_state — для відтворюваності

In [ ]:
model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=15,
    min_samples_leaf=2,
    random_state=42
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

Model Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#depth = 3 -> 78%
#depth = 10 -> 92%
#depth = 15 -> 93%
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
plt.figure(figsize=(25, 15))
plot_tree(model, feature_names = ['day_of_week_num',	'company_size_cat',	'industry_cat',
                                  'customer_tier_cat',	'region_cat',	'product_area_cat',
                                  'booking_channel_cat', 'reported_by_role_cat',	'customers_affected',
                                  'error_rate_pct',	'downtime_min',	'payment_impact_flag',	'security_incident_flag',	'data_loss_flag',
                                  'has_runbook',	'customer_sentiment_cat',	'description_length'],
          class_names=['Priority 1', 'Priority 2', 'Priority 3'], max_depth=3, filled = True, fontsize=10)
plt.show()

In [ ]:
model.get_params()

# **SVM**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import set_config
set_config(display='text')

In [ ]:
support_ticket_data.keys()

In [ ]:

df_support.info()

In [ ]:
df_support.head()

In [ ]:
df_support = support_ticket_data[['day_of_week_num', 'company_size_cat', 'industry_cat',
       'customer_tier_cat', 'region_cat', 'product_area_cat',
       'booking_channel_cat', 'reported_by_role_cat', 'customers_affected',
       'error_rate_pct', 'downtime_min', 'payment_impact_flag',
       'security_incident_flag', 'data_loss_flag', 'has_runbook',
       'customer_sentiment_cat', 'description_length']]
X_train, X_test, y_train, y_test = train_test_split(df_support, support_ticket_data['priority_cat'], test_size=0.30, random_state=101)

SVM дуже чутливий до масштабу ознак. StandardScaler() — найпоширеніший варіант для SVM

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

SVM model
Реалізація базується на бібліотеці libsvm

Найважливіші параметри SVC:

C=1.0 — коефіцієнт регуляризації. Чим більше C, тим менше регуляризації → модель намагається правильно класифікувати кожну точку (ризик переобучення). Чим менше C, тим більше регуляризації → спрощена межа, можливі помилки (краще узагальнення).

kernel='rbf' — ядро, яке використовується для побудови межі.

'linear' — лінійна межа; швидко, коли дані майже лінійно роздільні.
'rbf' (радіальне базисне) — універсальне, добре працює в більшості випадків.
gamma='scale' — впливає на “радіус впливу” однієї точки.

Маленьке gamma → широка межа (гладка, але може недонавчатись).
Велике gamma → межа «підганяється» під точки (ризик переобучення).
'scale' — стандартне значення, зазвичай добре працює.

In [ ]:
model = SVC(C=1, gamma = 1, kernel = 'rbf')

In [ ]:
model.fit(scaled_X_train,y_train)

In [ ]:
y_pred = model.predict(scaled_X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}

In [ ]:
grid = GridSearchCV(SVC(), param_grid, cv=5,refit=True)

# grid = GridSearchCV(
#     estimator=model,     # яку модель навчаємо (наприклад, SVC())
#     param_grid=params,   # словник з гіперпараметрами для перебору
#     cv=5                  # кількість фолдів у крос-валідації
# )

GridSearchCV(SVC(), param_grid, refit=True)

перебирає всі комбінації параметрів з param_grid;
автоматично проводить крос-валідацію для кожного набору параметрів;
обирає найкращу модель;
refit=True → після пошуку ще раз навчає SVC на всіх даних з найкращими параметрами.

In [ ]:
grid.fit(scaled_X_train,y_train)

In [ ]:
GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']})

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(scaled_X_test)

In [ ]:
print(confusion_matrix(y_test,grid_predictions))

In [ ]:
print(classification_report(y_test,grid_predictions))

# **Random Forest**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display="text")

In [ ]:
print("Розмір датасету:", X.shape)       # (70000, 784)
print("Кількість зображень:", len(X))   # 70000
print("Кількість ознак (пікселів):", X.shape[1])  # 784 = 28*28
print("Унікальні класи:", np.unique(y)) # 0–9